<a href="https://colab.research.google.com/github/sokrypton/ColabDesign/blob/v1.1.1/af/examples/af_pseudo_diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#AF_pseudo_diffusion + proteinMPNN
Hacking AlphaFold to be a diffusion model (for backbone generation). At each step add logits from proteinMPNN.


**WARNING**: This notebook is experimental, not validated or tested.

In [59]:
#@title setup
%%time
import os
if not os.path.isdir("params"):
  # get code
  os.system("pip -q install git+https://github.com/sokrypton/ColabDesign.git@v1.1.1")
  # for debugging
  os.system("ln -s /usr/local/lib/python3.7/dist-packages/colabdesign colabdesign")
  # download params
  os.system("mkdir params")
  os.system("apt-get install aria2 -qq")
  os.system("aria2c -q -x 16 https://storage.googleapis.com/alphafold/alphafold_params_2022-03-02.tar")
  os.system("tar -xf alphafold_params_2022-03-02.tar -C params")

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
from colabdesign import mk_afdesign_model, clear_mem
from colabdesign.mpnn import mk_mpnn_model
from IPython.display import HTML
from google.colab import files
import numpy as np

CPU times: user 544 µs, sys: 0 ns, total: 544 µs
Wall time: 521 µs


In [46]:
LENGTH = 100
clear_mem()
af_model = mk_afdesign_model(protocol="hallucination", use_templates=True)
af_model.prep_inputs(length=LENGTH)
mpnn_model = mk_mpnn_model()

In [93]:
af_model.restart(mode="gumbel")
L = af_model._len
af_model._inputs["batch"] = {"aatype":np.zeros(L).astype(int),
                             "all_atom_mask":np.zeros((L,37)),
                             "all_atom_positions":np.zeros((L,37,3))}

STEPS = 50 # might be overkill
BETA = 0.9
for k in range(STEPS):

  # add noise
  noise_weight = 1 - k/STEPS
  af_model._inputs["batch"]["all_atom_positions"] += np.random.normal(size=(L,37,3)) * noise_weight

  # denoise
  aux = af_model.predict(return_aux=True, verbose=False)
  af_model._inputs["batch"]["aatype"] = af_model._params["seq"].argmax(-1)[0]
  af_model._inputs["batch"]["all_atom_mask"][:,:4] = 1
  af_model._inputs["batch"]["all_atom_positions"] = af_model.aux["atom_positions"]

  # add logits from proteinmpnn at each stage
  mpnn_model.get_af_inputs(af_model)
  mpnn_out = mpnn_model.score()
  aux["log"]["mpnn"] = mpnn_out["score"]
  af_model._params["seq"] = BETA * af_model._params["seq"] + (1-BETA) * mpnn_out["logits"][:,:20]

  # save results
  af_model._save_results(aux)
  af_model._k += 1

1 models [0] recycles 0 hard 1 soft 0 temp 1 loss 3.88 con 3.88 plddt 0.33 ptm 0.23 mpnn 2.66
2 models [0] recycles 0 hard 1 soft 0 temp 1 loss 3.68 con 3.68 plddt 0.35 ptm 0.25 mpnn 2.64
3 models [0] recycles 0 hard 1 soft 0 temp 1 loss 3.58 con 3.58 plddt 0.36 ptm 0.25 mpnn 2.73
4 models [0] recycles 0 hard 1 soft 0 temp 1 loss 3.59 con 3.59 plddt 0.39 ptm 0.27 mpnn 2.53
5 models [0] recycles 0 hard 1 soft 0 temp 1 loss 3.56 con 3.56 plddt 0.40 ptm 0.27 mpnn 2.32
6 models [0] recycles 0 hard 1 soft 0 temp 1 loss 3.60 con 3.60 plddt 0.40 ptm 0.29 mpnn 2.30
7 models [0] recycles 0 hard 1 soft 0 temp 1 loss 3.38 con 3.38 plddt 0.48 ptm 0.37 mpnn 2.20
8 models [0] recycles 0 hard 1 soft 0 temp 1 loss 3.09 con 3.09 plddt 0.56 ptm 0.47 mpnn 2.04
9 models [0] recycles 0 hard 1 soft 0 temp 1 loss 3.04 con 3.04 plddt 0.59 ptm 0.49 mpnn 1.87
10 models [0] recycles 0 hard 1 soft 0 temp 1 loss 2.99 con 2.99 plddt 0.61 ptm 0.49 mpnn 1.87
11 models [0] recycles 0 hard 1 soft 0 temp 1 loss 2.99 con

In [94]:
af_model.plot_pdb()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [95]:
af_model.save_pdb(f"tmp.pdb")
af_model.get_seqs()

['APVVLRIRGDGAPRPPSPLSPETQAALKAFLAALGVTLPEGWVPYIDTSRGPVTVTPTELTLDLVGTGVPPLVFRLPDTTAELVEGDPDNTVVLTRRPPP']

In [96]:
HTML(af_model.animate(dpi=100))